# Introduction
What is the objective of this notebook? </br>
The objective of this notebook is to create a scipt which can make a repository of recipes and their ingredients. </br>

Below i will import the necessary libraries

In [1]:
import pandas as pd
import os
from datetime import datetime
from tqdm import tqdm
import regex as re
import time
import numpy as np
import random
import pyperclip as py
import openai 
import os
import pandas as pd
# from    dotenv import load_dotenv
# load_dotenv()
# openai.api_type     = os.getenv('OPENAI_TYPE')
# openai.api_base     = os.getenv('OPENAI_BASE')
# openai.api_version  = os.getenv('OPENAI_VERSION')
# openai.api_key      = os.getenv('OPENAI_KEY')
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin
from tqdm import tqdm

import helper as helper

In [87]:
# Load the recipes data 
recipes = pd.read_csv('recipes.csv')
# Load the ingredients data
ingredients = pd.read_csv('ingredients_df.csv')
# Load the pantry data
pantry = pd.read_csv('pantry.csv')
# Load the conversion data
conversion = pd.read_csv('conversion.csv')


''

In [62]:
def urls_on_same_domain(url, max_urls=1000):
    """
    Counts the number of unique URLs on the same domain as the given URL,
    iterating through links until no more are found or a max limit is reached.
    The progress is displayed using tqdm.

    Args:
    url (str): The starting URL to fetch and analyze.
    max_urls (int): Maximum number of URLs to fetch.

    Returns:
    int: The total number of unique URLs found on the same domain.
    """
    visited_urls = set()
    urls_to_visit = set([url])
    domain = urlparse(url).netloc

    with tqdm(total=max_urls, desc=f"Analyzing URLs currently found {len(visited_urls)}, and have {len(urls_to_visit)} to go", unit="url") as pbar:
        while len(urls_to_visit) != 0 and len(urls_to_visit) < max_urls:
            current_url = urls_to_visit.pop()
            try:
                response = requests.get(current_url)
                response.raise_for_status()

                soup = BeautifulSoup(response.content, 'html.parser')
                anchors = soup.find_all('a', href=True)

                for anchor in anchors:
                    href = anchor['href']
                    full_url = urljoin(current_url, href)
                    href_domain = urlparse(full_url).netloc

                    if href_domain == domain and full_url not in visited_urls and '#comment' not in full_url and '#respond' not in full_url:
                        urls_to_visit.add(full_url)

                visited_urls.add(current_url)
                pbar.update(1)
                pbar.set_description(f"Analyzing URLs currently found {len(visited_urls)}, and have {len(urls_to_visit)} to go")
            except requests.RequestException:
                # Handle exceptions for requests
                continue

    return visited_urls, urls_to_visit

def get_text_from_url(url):
    """
    Fetches the content from a given URL and returns the text separated by lines.
    If the URL gets redirected more than twice, returns 'redirect link'.

    Args:
    url (str): The URL from which to fetch the content.

    Returns:
    str: A string containing the text from the URL, separated by lines, or 'redirect link'.
    """
    # Send a GET request to the URL with redirection allowed
    response = requests.get(url, allow_redirects=True)
    
    # Check the number of redirects
    if len(response.history) > 2:
        return 'redirect link'
    
    # Raise an exception if the request was unsuccessful
    response.raise_for_status()

    # Parse the content of the request with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract and return the text, separated by lines
    return '\n'.join(soup.stripped_strings)

def ingredients_cleaner(text):
    try:
        text = text.split('Ingredienser')[1]
        text = text.split('Udskriv')[0]
        text = text.strip()
    except:
        text = ''
    if 'Archive' in text:
        text = ''
    return text

def set_individual_price(row):
    amount = row['amounts']
    purchase_size = row['purchase_size']
    price = row['price']
    input_units = row['units']
    purchase_unit = row['purchase_unit']
    if len(conversion[(conversion.units == input_units) & (conversion.purchase_unit == purchase_unit)] ) > 0:
        faktor = conversion[(conversion.units == input_units) & (conversion.purchase_unit == purchase_unit)].rate.values[0]
    else:
        return 0

    if purchase_size == 0:
        return 0
    else:
        if input_units == purchase_unit:
            y = np.ceil(amount / purchase_size) * price if amount > purchase_size else price
        else:
            if input_units == 'g' and purchase_unit == 'stk':
                y = price
            else:
                # we need to make it so that it looks in the conversion table and then returns the 
                y = np.ceil(amount * faktor / purchase_size) * price  if amount > purchase_size else price
    
    # lets also round the product of faktor by purchase_size and amount
    z = np.ceil(amount / purchase_size) * faktor if amount > purchase_size else faktor * amount
    
    return y

def find_dinner(test_text, test_url):
    segment = test_text.split(test_url)[1]
    if 'Aftensmad' in segment:
        if 'Tilbehør Aftensmad' in segment:
            return False
        return True
    else:
        return False

def find_vegeterian(test_text, test_url):
    segment = test_text.split(test_url)[1]
    if 'Vegetar' in segment:
        return True
    else:
        return False

def find_addon(test_text, test_url):
    segment = test_text.split(test_url)[1]
    if 'Tilbehør Aftensmad' in segment:
        return True
    else:
        return False

def ing_extract_measurement(line, measurements):
    """This function extracts the measurement from a line of text, from the ingredients_df

    Args:
        line (str): ingredients line
        measurements (list): list of measurements

    Returns:
        str: measurement
    """
    local_measurement = ''

    for measurement in measurements:
        if f'{measurement} ' in line:
            return measurement
        
    return local_measurement

def implied_terms(ingredient):
    if 'øko' in ingredient:
        ingredient = ingredient.replace('øko', '')

    return ingredient

def extract_measurement(line, measurements):
    local_measurement = ''

    for measurement in measurements:
        if f' {measurement} ' in line:
            return measurement
        
    return local_measurement

def extract_amount(line):
    pattern = '^\d+'
    match = re.findall(pattern, line)
    if len(match) > 0:
        return match[0]
    else:
        return 0

def extract_ingredient(line, measurement):
    """This function extracts the ingredient from a line of text

    Args:
        line (str): a line of text containg an ingredient, amount and measurement
        measurement (list): a list of measurements

    Returns:
        amount: the amount of the ingredient
        measurement: the measurement of the ingredient
        new_line: the ingredient
    """
    amount = extract_amount(line)
    local_measurement = extract_measurement(line, measurement)
    if local_measurement != '':
        new_line = line.replace(amount, '')''
        new_line = new_line.replace(f' {local_measurement} ', '')
        new_line = new_line.strip()
    else:
        new_line = line.replace(amount, '')
        new_line = new_line.strip()

    # now lets clean up after the measurement
    if ',' in amount:
        amount = amount.replace(',', '.')
        new_line = new_line.replace(amount, '')
        new_line = new_line.strip()
    else:
        new_line = new_line.replace(amount, '')
        new_line = new_line.strip()

    # now lets remove the implied terms
    new_line = implied_terms(new_line)
    new_line = new_line.strip()

    return amount, local_measurement, new_line

def ingredients_setter(ingredients_text, url, measurements, pantry):
    """This function takes a string of ingredients and sets them in the pantry"""
    ingredients_text = ingredients_cleaner(ingredients_text)    

    for line in ingredients_text.splitlines():
        amount, local_measurement, new_line = extract_ingredient(line, measurements)
        pantry.loc[len(pantry)] = [url, amount, local_measurement, new_line]

    return pantry

def get_measurements(conversion):
    """This function returns a list of all the measurements that we have in the conversion table

    Args:
        conversion (DataFrame): the conversion dataframe 

    Returns:
        list: a list of all the measurements that we have in the conversion table
    """
    # weird measurements
    measurements = ['drys', 'dryp', 'skiver', 'skive', 'stængel', 'stængler', 'ark', 'dåser', 'dåse', 'pakke', 'pakker', 'håndfulde', 'håndfuld']

    # now lets get all the measurements from the conversion table, and add them to the measurements list
    for measurement in conversion.units.unique():
        measurements.append(measurement)

    return measurements

measurements = get_measurements(conversion)


In [60]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin
from tqdm import tqdm


def urls_on_same_domain(url, max_urls=1000):
    """
    Counts the number of unique URLs on the same domain as the given URL,
    iterating through links until no more are found or a max limit is reached.
    The progress is displayed using tqdm.

    Args:
    url (str): The starting URL to fetch and analyze.
    max_urls (int): Maximum number of URLs to fetch.

    Returns:
    int: The total number of unique URLs found on the same domain.
    """
    visited_urls = set()
    urls_to_visit = set([url])
    domain = urlparse(url).netloc

    # Common user-agent string of a web browser
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    with tqdm(total=max_urls, desc=f"Analyzing URLs currently found {len(visited_urls)}, and have {len(urls_to_visit)} to go", unit="url") as pbar:
        while len(urls_to_visit) != 0 and len(visited_urls) < max_urls:
            current_url = urls_to_visit.pop()
            try:
                response = requests.get(current_url, headers=headers)
                response.raise_for_status()

                soup = BeautifulSoup(response.content, 'html.parser')
                anchors = soup.find_all('a', href=True)

                for anchor in anchors:
                    href = anchor['href']
                    full_url = urljoin(current_url, href)
                    href_domain = urlparse(full_url).netloc

                    if href_domain == domain and full_url not in visited_urls and '#comment' not in full_url and '#respond' not in full_url:
                        urls_to_visit.add(full_url)

                visited_urls.add(current_url)
                pbar.update(1)
                pbar.set_description(f"Analyzing URLs currently found {len(visited_urls)}, and have {len(urls_to_visit)} to go")
            except requests.RequestException:
                # Handle exceptions for requests
                continue

    return len(visited_urls)

def get_text_from_url(url):
    """
    Fetches the content from a given URL and returns the text separated by lines.
    If the URL gets redirected more than twice, returns 'redirect link'.

    Args:
    url (str): The URL from which to fetch the content.

    Returns:
    str: A string containing the text from the URL, separated by lines, or 'redirect link'.
    """
    # Common user-agent string of a web browser
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    # Send a GET request to the URL with the user-agent header and redirection allowed
    response = requests.get(url, headers=headers, allow_redirects=True)
    
    # Check the number of redirects
    if len(response.history) > 2:
        return 'redirect link'
    
    # Raise an exception if the request was unsuccessful
    response.raise_for_status()

    # Parse the content of the request with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract and return the text, separated by lines
    return '\n'.join(soup.stripped_strings)

url = 'https://vegetariskhverdag.dk/opskrifter/'

test = get_text_from_url(url)

print(test.split('Ingredienser')[1])


I OVNEN
3/4 butternut squash
1 pakke halloumi (200-250 g)
4 fed hvidløg
2-3 spsk olie
1 tsk salt
Peber
SYLTET SQUASH
1/4 butternut squash
1 dl sukker
1 dl vand
2 dl eddike (æblecider eller lign.)
1 tsk salt
LIMA HUMMUS
1 dåse drænede limabønner
0,5-0,75 tsk salt
1 spsk græskar-syltelage
1 spsk kalamata lage
1 spsk olivenolie
DRESSING
De 4 bagte hvidløgsfed fra ovnen
1 tsk salt
0,5 dl olivenolie
2-3 spsk eddike
1 tsk dijonsennep
Sort peber
DESUDEN
1 hoved radicchio salat
En god håndfuld tomater (kan udelades, jeg havde bare ved hånden)
1 håndfuld kalamataoliven
Et lille bundt koriander (eller valgfri grøn krydderurt)
Sådan gør du
Sæt ovnen på 225 grader varmluft.
Flæk butternut squashen på langs, fjern indmaden og skær 3/4 i 1-1,5 cm skiver. Gem den sidste fjerdedel til senere. Behold skallen på.
Pil hvidløgsfeddene og læg hele på bagepladen.
Del halloumi osten i 4 kvarte på langs og placer på bagepladen.
Giv det hele et godt drys olivenolie, salt og peber.
Bland godt og bag i 20 minut

In [91]:
def get_measurements(conversion):
    """This function returns a list of all the measurements that we have in the conversion table

    Args:
        conversion (DataFrame): the conversion dataframe 

    Returns:
        list: a list of all the measurements that we have in the conversion table
    """
    # weird measurements
    measurements = ['drys', 'dryp', 'skiver', 'skive', 'stængel', 'stængler', 'ark', 'dåser', 'dåse', 'pakke', 'pakker', 'håndfulde', 'håndfuld']

    # now lets get all the measurements from the conversion table, and add them to the measurements list
    for measurement in conversion.units.unique():
        measurements.append(measurement)

    return measurements

measurements = get_measurements(conversion)

get_measurements(conversion)

removeable = ['drys', 'dryp', 'skiver', 'skive', 'stængel', 'stængler', 'ark', 'dåser', 'dåse', 'pakke', 'pakker', 'håndfulde', 'bakke', 'bakker']
# removeable = ['håndfulde', 'håndfuld']

for ingredient in ingredients.ingredients.unique():
    local_measurement = ing_extract_measurement(ingredient, measurements)
    new_ingredient = ''
    if local_measurement in removeable:
        new_ingredient = ingredient.replace(f'{local_measurement} ', '')
        new_ingredient = new_ingredient.strip()

        if new_ingredient != '':
            print(ingredient, new_ingredient, local_measurement)
            ingredients.loc[ingredients.ingredients == ingredient, 'ingredients'] = new_ingredient
            # now lets make sure that we set the unit to grams
            # pantry.loc[pantry.ingredients == ingredient, 'units'] = 'g'
            # now lets multiply the amount by 25 grams
            # pantry.loc[pantry.ingredients == ingredient, 'amounts'] = pantry.loc[pantry.ingredients == new_ingredient, 'amounts'] * 25
            pantry.loc[pantry.ingredients == ingredient, 'ingredients'] = new_ingredient

        else:
            print('error',ingredient)

        

stængler bladselleri bladselleri stængler
skiver bacon bacon skiver
stængler frisk rosmarin frisk rosmarin stængler
dåser hakkede tomater hakkede tomater dåser
stængler frisk timian frisk timian stængler
skiver brød brød skiver
ark husblas husblas ark
skiver rugbrød rugbrød skiver
stængel frisk rosmarin frisk rosmarin stængel
drys muskatnød muskatnød drys
stængler forårsløg forårsløg stængler
drys sesamfrø sesamfrø drys
stængel citrongræs citrongræs stængel
stængler rabarber rabarber stængler
skiver ingefær ingefær skiver
skiver agurk agurk skiver
dåser kidneybønner kidneybønner dåser
skiver cheddar ost cheddar ost skiver
drys salt salt drys
dryp olivenolie olivenolie dryp
stængler persille persille stængler
drys frisk timian frisk timian drys
skive brød brød skive
stængel frisk mynte frisk mynte stængel
stængel frisk timian frisk timian stængel
skiver ost ost skiver
stængler frisk mynte frisk mynte stængler
skiver landskinke landskinke skiver
dryp sriracha chilisauce sriracha chilisau

In [95]:
# now lets walk through the pantry and make sure that each time we find 'håndfulde' we will multiply the amount by 25 grams, and set the unit to grams
for ingredient in pantry.ingredients.unique():
    if 'håndfulde' in ingredient:
        pantry.loc[pantry.ingredients == ingredient, 'units'] = 'g'
        pantry.loc[pantry.ingredients == ingredient, 'amounts'] = pantry.loc[pantry.ingredients == ingredient, 'amounts'] * 25
        pantry.loc[pantry.ingredients == ingredient, 'ingredients'] = ingredient.replace('håndfulde', '').strip()
    if 'håndfuld' in ingredient:
        pantry.loc[pantry.ingredients == ingredient, 'units'] = 'g'
        pantry.loc[pantry.ingredients == ingredient, 'amounts'] = pantry.loc[pantry.ingredients == ingredient, 'amounts'] * 25
        pantry.loc[pantry.ingredients == ingredient, 'ingredients'] = ingredient.replace('håndfuld', '').strip()

# now lets check if we did it right
pantry[pantry.ingredients.str.contains('håndfuld')]

,urls,amounts,units,ingredients,purchase_unit,spice,price,purchase_size,local_price,pareto,in_recipe,matches


In [100]:
# now lets reset the urls count in the ingredients dataframe
ingredients['urls'] = 0
# now lets for each of the ingredients, lets count the number of urls that we have
for ingredient in tqdm(ingredients.ingredients.unique()):
    urls = len(pantry[pantry.ingredients == ingredient].urls.unique())
    ingredients.loc[ingredients.ingredients == ingredient, 'urls'] = urls

# now lets see how many which are now set to 0
ingredients[ingredients.urls == 0]

  0%|          | 0/715 [00:00<?, ?it/s]

100%|██████████| 715/715 [00:01<00:00, 567.06it/s]


,ingredients,pareto,purchase_unit,type,price,purchase_size,count,salad_average,veggie_average,urls
17,0sbouillon,True,g,0.0,45.0,200.0,158,NaN,NaN,0
130,kogende vand,True,NaN,1.0,NaN,NaN,27,NaN,NaN,0
164,koldt vand,True,NaN,1.0,NaN,NaN,19,NaN,NaN,0
200,stuetempereret vand,True,NaN,NaN,NaN,NaN,14,NaN,NaN,0
347,five 1,False,NaN,NaN,NaN,NaN,5,NaN,NaN,0


In [110]:
ingredients = pd.read_clipboard()
ingredients

,ingredients,pareto,purchase_unit,type,price,purchase_size,salad_averaeg,veggie_average,count
0,olivenolie,True,ml,long,160.0,1000.0,NaN,NaN,499
1,smør,True,g,long,26.0,200.0,37.500000,24.435484,495
2,sukker,True,g,long,25.0,1000.0,126.944444,81.153846,477
3,hvidløg,True,fed,long,30.0,12.0,0.876543,1.140071,446
4,æg,True,stk,short,36.0,10.0,0.812500,1.406250,392
...,...,...,...,...,...,...,...,...,...
740,ristede løg,False,NaN,NaN,NaN,NaN,NaN,NaN,1
741,rismel,False,NaN,NaN,NaN,NaN,NaN,NaN,1
742,julesalat,False,NaN,NaN,NaN,NaN,NaN,NaN,1
743,kageplast,False,NaN,NaN,NaN,NaN,NaN,NaN,1


In [102]:
# now lets remove any any duplicates in the ingredients and make sure that we keep the first one of each
ingredients.drop_duplicates(subset=['ingredients'], keep='first', inplace=True)

In [111]:
ingredients.to_csv('ingredients_df.csv', index=False)